# IBM Applied Data Science Capstone Project

## Project1
Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.


2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.


6. Use the .shape method to print the number of rows of your dataframe.

## Project2 
Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

We will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [1]:
#Import Libraries

import numpy as np
import pandas as pd

import json
import urllib.request

import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

#!conda install -c conda-forge geopy --yes

import bs4 as bs
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans


### Scrap data from Wikipedia page into a DataFrame

In [2]:

import geopy

# send the GET request
#data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(source,'html.parser')

#
table = soup.find('table')
table_rows = table.find_all('tr')

# create lists to store data
list_data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        list_data.append(row)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [3]:
df = pd.DataFrame(list_data, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [4]:
# Cells that have an assigned borough. 
#And Ignore cells with a borough that is 'Not assigned'.

df_dropna = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### More than one neighborhood can exist in one postal code area. 
### For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
# More than one neighborhood can exist in one postal code area.

df_grouped = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
#Group
df_grouped.loc[df_grouped['Neighbourhood']=='Not assigned', ['Neighbourhood']]
df_grouped.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Show the data

In [7]:
# Represent the data
df1 = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood"])

test_data_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_data_list:
    df1 = df1.append(df_grouped[df_grouped["PostalCode"]==postcode], ignore_index=True)
    
df1

,PostalCode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


 
 ### Use the .shape method to print the number of rows of your dataframe.
 

In [8]:
# Print the number of rows of the cleaned dataframe
df_grouped.shape


(103, 3)

### Getting the latitude and the longitude coordinates of each neighborhood.
The geopy/geocoder package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [10]:
# Retrieve coordinates from the csv file on
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")
coordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# In csv file one column name is different from the other file
#So replace or rename it
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

# Combine both the tables
df_new = df_grouped.merge(coordinates, on="PostalCode", how="left")
df_new.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
# Represent the data
df1 = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood","Latitude", "Longitude"])

test_data_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_data_list:
    df1 = df1.append(df_new[df_new["PostalCode"]==postcode], ignore_index=True)
    
df1

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420
